### INPUT Number of rows for aggregated dataset

In [45]:
grouped = 100

In [46]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import os

# List all files under the input directory
#path = '/kaggle'
path = './'
input_path  = path + '/input/gearbox-fault-diagnosis/'
output_path = path + '/input/gearbox-fault-diagnosis-stdev/'

# Read datasets
## Healthy gearbox

In [47]:
h30hz0  = pd.read_csv(input_path + "/Healthy/h30hz0.csv")
h30hz10 = pd.read_csv(input_path + "/Healthy/h30hz10.csv")
h30hz20 = pd.read_csv(input_path + "/Healthy/h30hz20.csv")
h30hz30 = pd.read_csv(input_path + "/Healthy/h30hz30.csv")
h30hz40 = pd.read_csv(input_path + "/Healthy/h30hz40.csv")
h30hz50 = pd.read_csv(input_path + "/Healthy/h30hz50.csv")
h30hz60 = pd.read_csv(input_path + "/Healthy/h30hz60.csv")
h30hz70 = pd.read_csv(input_path + "/Healthy/h30hz70.csv")
h30hz80 = pd.read_csv(input_path + "/Healthy/h30hz80.csv")
h30hz90 = pd.read_csv(input_path + "/Healthy/h30hz90.csv")

## Broken gearbox

In [48]:
b30hz0  = pd.read_csv(input_path + "/BrokenTooth/b30hz0.csv")
b30hz10 = pd.read_csv(input_path + "/BrokenTooth/b30hz10.csv")
b30hz20 = pd.read_csv(input_path + "/BrokenTooth/b30hz20.csv")
b30hz30 = pd.read_csv(input_path + "/BrokenTooth/b30hz30.csv")
b30hz40 = pd.read_csv(input_path + "/BrokenTooth/b30hz40.csv")
b30hz50 = pd.read_csv(input_path + "/BrokenTooth/b30hz50.csv")
b30hz60 = pd.read_csv(input_path + "/BrokenTooth/b30hz60.csv")
b30hz70 = pd.read_csv(input_path + "/BrokenTooth/b30hz70.csv")
b30hz80 = pd.read_csv(input_path + "/BrokenTooth/b30hz80.csv")
b30hz90 = pd.read_csv(input_path + "/BrokenTooth/b30hz90.csv")

## Function to compute aggregated features: standard deviation

In [49]:
def stdev_features(df, grouped, load, outcome):
    #### Create empty dataframe with columns a1,a2,a3,a4
    df_result = pd.DataFrame( [ np.zeros(len(df.columns)-2) ],columns= df.columns[:-2])

    #### Aggregate in groups of nrows computing the standard deviation
    # Remove load & failure columns, keeping only a1,a2,a3,a4
    stdev_lenght=len(df.columns)-2
    
    #### Compute number of rows of the aggregated dataframe
    nrows_raw = len(df.index)
    nrows = math.floor(nrows_raw / grouped)
    nrows_dropped = nrows_raw - nrows*grouped
    print("nrows_raw=", nrows_raw, "   nrows=", nrows, "   Number of dropped rows of grouped= ", nrows_dropped/grouped*100,"%\n")
    
    # Iterate every 'grouped' rows and compute stdev per column
    for i in range(nrows):
      df_group = df.iloc[i*grouped:i*grouped+grouped,:]
      df_stdev = pd.DataFrame(df_group.std()).transpose()
      # Remove load & failure columns
      df_stdev=df_stdev.iloc[:,:stdev_lenght]
      # Add row of calculated stdev
      df_result = df_result.append(df_stdev, ignore_index=True)

      #print ("i*grouped TO i*grouped+grouped", i*grouped, i*grouped+grouped)
      #print ("row, df_stdev=\n", row, df_stdev.iloc[:,:])
      #print ("df_result=\n", df_result)
      #print("row", i, "\n", df_group)


    # Remove the first row (it was the seed of zeros for initializing df_result)
    #print(df_result)
    df_result = df_result.iloc[1:,:]
    # Add the column for 'load'
    df_result['load'] = load*np.ones((nrows,1))

    # Add the column for 'failure'
    failure = np.ones((nrows,1))
    df_result['failure'] = outcome
    #print(df_result)
    
    return df_result

## Broken, Load 50%

In [50]:
# [H.5] DATASET BROKEN, 50% LOAD
load = 50

# CREATE THE COLUMN FOR 'load'
## Expressed as %
b30hz50['load'] = load

# CREATE THE COLUMN FOR 'failure'
##   0 is healthy
##   1 is broken
failure = 1
b30hz50['failure'] = failure

b30hz50_stdev= stdev_features (df = b30hz50, grouped= grouped, load= load, outcome= failure)
b30hz50_stdev.describe()

# Save to file
b30hz50_stdev.to_csv(output_path + 'broken30hz50.csv', index = False)

nrows_raw= 94208    nrows= 942    Number of dropped rows of grouped=  8.0 %



## Healthy, Load 50%

In [51]:
# [B.5] DATASET HEALTHY, 50% LOAD
load = 50
h30hz50['load'] = load
failure = 0
h30hz50['failure'] = failure

h30hz50_stdev= stdev_features (df = h30hz50, grouped= grouped, load= load, outcome= failure)
h30hz50_stdev.describe()

# Save to file
h30hz50_stdev.to_csv(output_path + 'healthy30hz50.csv', index = False)

nrows_raw= 110848    nrows= 1108    Number of dropped rows of grouped=  48.0 %



## Load  0%

In [52]:
# Broken dataset
load = 0
failure = 1
filename = 'broken30hz0.csv'
df = b30hz0

b30hz0['load'] = load
b30hz0['failure'] = failure
b30hz0_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz0_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 0
failure = 0
filename = 'healthy30hz0.csv'
df = h30hz0

h30hz0['load'] = load
h30hz0['failure'] = failure
h30hz0_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz0_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 88320    nrows= 883    Number of dropped rows of grouped=  20.0 %

nrows_raw= 88832    nrows= 888    Number of dropped rows of grouped=  32.0 %



# Load 90%

In [66]:
# Broken dataset
load = 90
failure = 1
filename = 'broken30hz90.csv'
df = b30hz90

b30hz90['load'] = load
b30hz90['failure'] = failure
b30hz90_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz90_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 90
failure = 0
filename = 'healthy30hz90.csv'
df = h30hz90

h30hz90['load'] = load
h30hz90['failure'] = failure
h30hz90_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz90_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 105728    nrows= 1057    Number of dropped rows of grouped=  28.000000000000004 %

nrows_raw= 106752    nrows= 1067    Number of dropped rows of grouped=  52.0 %



# Load 10%

In [54]:
# Broken dataset
load = 10
failure = 1
filename = 'broken30hz10.csv'
df = b30hz10

b30hz10['load'] = load
b30hz10['failure'] = failure
b30hz10_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz10_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 10
failure = 0
filename = 'healthy30hz10.csv'
df = h30hz10

h30hz10['load'] = load
h30hz10['failure'] = failure
h30hz10_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz10_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 111616    nrows= 1116    Number of dropped rows of grouped=  16.0 %

nrows_raw= 92928    nrows= 929    Number of dropped rows of grouped=  28.000000000000004 %



# Load 20%

In [55]:
# Broken dataset
load = 20
failure = 1
filename = 'broken30hz20.csv'
df = b30hz20

b30hz20['load'] = load
b30hz20['failure'] = failure
b30hz20_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz20_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 20
failure = 0
filename = 'healthy30hz20.csv'
df = h30hz20

h30hz20['load'] = load
h30hz20['failure'] = failure
h30hz20_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz20_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 114432    nrows= 1144    Number of dropped rows of grouped=  32.0 %

nrows_raw= 108544    nrows= 1085    Number of dropped rows of grouped=  44.0 %



# Load 30%

In [56]:
# Broken dataset
load = 30
failure = 1
filename = 'broken30hz30.csv'
df = b30hz30

b30hz30['load'] = load
b30hz30['failure'] = failure
b30hz30_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz30_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 30
failure = 0
filename = 'healthy30hz30.csv'
df = h30hz30

h30hz30['load'] = load
h30hz30['failure'] = failure
h30hz30_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz30_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 89856    nrows= 898    Number of dropped rows of grouped=  56.00000000000001 %

nrows_raw= 106240    nrows= 1062    Number of dropped rows of grouped=  40.0 %



# Load 40%

In [57]:
# Broken dataset
load = 40
failure = 1
filename = 'broken30hz40.csv'
df = b30hz40

b30hz40['load'] = load
b30hz40['failure'] = failure
b30hz40_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz40_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 40
failure = 0
filename = 'healthy30hz40.csv'
df = h30hz40

h30hz40['load'] = load
h30hz40['failure'] = failure
h30hz40_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz40_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 94464    nrows= 944    Number of dropped rows of grouped=  64.0 %

nrows_raw= 100608    nrows= 1006    Number of dropped rows of grouped=  8.0 %



# Load 60%

In [58]:
# Broken dataset
load = 60
failure = 1
filename = 'broken30hz60.csv'
df = b30hz60

b30hz60['load'] = load
b30hz60['failure'] = failure
b30hz60_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz60_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 60
failure = 0
filename = 'healthy30hz60.csv'
df = h30hz60

h30hz60['load'] = load
h30hz60['failure'] = failure
h30hz60_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz60_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 95488    nrows= 954    Number of dropped rows of grouped=  88.0 %

nrows_raw= 99840    nrows= 998    Number of dropped rows of grouped=  40.0 %



# Load 70%

In [59]:
# Broken dataset
load = 70
failure = 1
filename = 'broken30hz70.csv'
df = b30hz70

b30hz70['load'] = load
b30hz70['failure'] = failure
b30hz70_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz70_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 70
failure = 0
filename = 'healthy30hz70.csv'
df = h30hz70

h30hz70['load'] = load
h30hz70['failure'] = failure
h30hz70_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz70_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 100864    nrows= 1008    Number of dropped rows of grouped=  64.0 %

nrows_raw= 101376    nrows= 1013    Number of dropped rows of grouped=  76.0 %



# Load 80%

In [60]:
# Broken dataset
load = 80
failure = 1
filename = 'broken30hz80.csv'
df = b30hz80

b30hz80['load'] = load
b30hz80['failure'] = failure
b30hz80_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
b30hz80_stdev.to_csv(output_path + filename, index = False)

# Healthy dataset
load = 80
failure = 0
filename = 'healthy30hz80.csv'
df = h30hz80

h30hz80['load'] = load
h30hz80['failure'] = failure
h30hz80_stdev = stdev_features (df = df, grouped= grouped, load= load, outcome= failure)
h30hz80_stdev.to_csv(output_path + filename, index = False)

nrows_raw= 110335    nrows= 1103    Number of dropped rows of grouped=  35.0 %

nrows_raw= 99840    nrows= 998    Number of dropped rows of grouped=  40.0 %

